In [ ]:
c:java.lang.String = """    <?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleDevelopmentRegion</key>
    <string>en_US</string>
    <key>CFBundleExecutable</key>
    <string>${executable}</string>
    <key>CFBundleIconFile</key>
    <string>icon.icns</string>
    <key>CFBundleIdentifier</key>
    <string>org.scala-native.${bundleName}</string>
    <key>CFBundleInfoDictionaryVersion</key>
    <string>6.0</string>
    <key>CFBundlePackageType</key>
    <string>APPL</string>
    <key>CFBundleSignature</key>
    <string>????</string>
    <key>NSHumanReadableCopyright</key>
    <string>${copyright}</string>
    ${infoPlistExtras}
    </dict>
    </plist>"""

val infoPlist = TaskKey[Path]("infoplist")

lazy val root = Project(id, base)
    .enablePlugins(ScalaNativePlugin)
    .settings(
        scalaVersion := scala213,
        crossScalaVersions := Seq(scala213),
        organization := "org.scala-native",
        name := id,
        scalacOptions ++= List("-opt:speed", "-g"),
        nativeLinkStubs := true,
        nativeLinkMode := LinkMode.Dynamic
        /* FIXME: Enabling this breaks the build on CI for some reason */
        // nativeClangArgs += s"-I${base}/src/main/c"
    )

/* Additional settings that are conditionally added by other projects */
def addSettingsIf(cond: Boolean)(ss: Def.Setting[_]*): Def.Initialize[Seq[Def.Setting[_]]] =
    if (cond) ss else Nil

addCommandAlias("packageMac", "+rootMac package")

lazy val jni = project
    .in(file("jni"))
    .dependsOn(root % "test->test;compile->compile")
    .settings(
        moduleName := s"$id-jni",
        nativeBuildSystem := BuildSystemJNI,
        unmanagedSourceDirectories in Compile += base / "include",
        sourceDirectory in Javadoc := base / "java",
        javacOptions in compile ++= Seq("-Xlint:deprecation", "-Xdoclint:none"),
        javaHome <<= target map { t => " -Xmx5G -Djava.home=" + (t / "classes") },
        javadocOptions ++= Seq("-quiet", "-windowtitle", title),
        libraryDependencies += "javax.annotation" % "jsr250-api" % "1.0",
        addSettingsIf(!sys.props.contains("CI"))(
            scalacOptions += "-Ywarn-unused-import",
            scalafmtOnCompile.in    (ThisBuild) := false,
            scalafmtOnFormat.in     (ThisBuild) := false,
            scalafmtCheck.in        (ThisBuild) := false,
            scalafmtSbtCheck.in     (ThisBuild) := false
        ),
        publishArtifact := false
    )

val rootShared = project
    .in(file("shared"))
    .enablePlugins(ScalaNativePlugin)
    .aggregate(macos, linux)
    .settings
        sharedSettings,
        crossProjectSettings,
        releasePublishBoth,
        name := id,
        version := v,
        scalaVersion := sv,
        buildInfoKeys := buildInfoKeySet,
        buildInfoPackage := pkg,
        buildInfoObject := "buildinfo",
        buildInfoKeys += BuildTime,
        buildInfoKeys += ("scalaVersion" -> sys.props.getOrElse("SCALA_VERSION", scalaVersion.value))

lazy val macos = rootShared.jsConfigure(_.enablePlugins(ScalaJSWebjarPlugin)).
    jsPlatform(true).
    settings(osSetting.value.platform.settings: _*)

lazy val linux = rootShared.jvmConfigure(_.enablePlugins(sbtassembly.AssemblyPlugin)).
                    jvmPlatform(false).
                    settings(linuxOsSetting.value.platform.settings: _*).
                    dependsOn(rootShared)

//------------------------------------------------------------------------------

lazy val core = project.
    in(file("core")).
    enablePlugins(BuildInfoPlugin).
    dependsOn(rootShared % "test->test").
    settings(sharedSettings ++ buildInfoSettings :+
        name := "nscp-core",
        libraryDependencies ++= Seq(
            "org.scalameta" %% "munit"   % V.munit % Test,
            "org.scalameta" %% "munit-scalacheck"  % V.munit % Test
        ),
        testFrameworks += new TestFramework("munit.MunitSuite"),
        // Disable parallel execution of MUnit tests because it causes false positives (e.g., https://github.com/lianch
        // TODO: Remove this once https://github.com/scalameta/munit/pull/206 is merged and released.
        scalacOptions -= "-Xfatal-warnings",
        assembly / mainClass := Some("codes.quine.labo.recheck.RecheckApp"),
        buildInfoKeys += ("version" -> Version(version = version.value)),
        packageDoc / publishArtifact := false,
        releaseCrossTarget := true
    )

val `cli-native` = os.baseDir / "cli" / "native"
val `cli-cross` = os. baseDir / "cli" / "cross"

lazy val cli = (project.in(`cli-native`) +: crossProject.from(`cli-cross`, `cli-native`).
    settings(sharedSettings ++ osSetting.value.cross.settings: _): CrossProject).
    jsConfigure(_.enablePlugins(ScalaJSWebjarPlugin)).
    jvmConfigure(_.enablePlugins(JavaAppPackaging)).
    settings(name := "nscp-cli",
                libraryDependencies ++= Dependencies. libraries.map(_ % Test),
                sourceDirectory := (`cli-native` / "src") * (osSetting.value == Os.Name.MacOS),
                nativeSourceDirectory := (`cli-native` / "src") * (osSetting.
                                                                    value != Os.Name.MacOS),
                crossSources := (sourceDirectory.value ** "*.*").get(),
                nativeCliPackageBin := {
                    if ((osSetting.value == Os.Name.Windows).isDefined)
                        sys.error("Native CLI sources are not available on Windows.")
                    else
                        nativeSourceDirectory.value
                },
                nativeCliPackageSrc := {
                    if ((osSetting.value == Os.Name.Windows).isDefined)
                        sourceDirectory.value
                    else
                        sys.error("Source files for the native CLI are not available on non-Windows platforms.")
                        sys.error("Source files for the native CLI are not available on other platforms.")
                }
            ).
            jvm.
            enablePlugins(BuildInfoPlugin).
            settings(buildInfoKeys := buildInfoKeys.value.filterNot(_ == "nativeCliPackageSrc"))
            
            lazy val jsvm = cli.js.
                enablePlugins(ScalaJSBundlerPlugin).
                settings(npmDevDependencies in Compile ++= Dependencies.devNpmDeps.value,
                        webpackBundlingMode := BundlingMode.LibraryOnly,
                        bundleFile <<= (target,
                                        moduleKind in (Compile, scalaJSPipeline),
                                        mainClass in (Compile, packageBin)) map {
                            case (t, mk, mc) => t / s"${mk.capitalized}Main.js" -> m / s"${mc.capitalized}${if (mk == ModuleKind.`no
                            case (t, m, c) => t / s"${c.getcapitalized(true)}.js" -> m / s"${c.getcapitalized}.js"
                                            (t, m, c) => t / s"${c.getSimpleName}.js"
                                        })
    
            lazy val jvmmacro = cli.jvm.
                dependsOn(jsvm).
                enablePlugins(DetektPlugin).
                settings(libraryDependencies += Dependencies.libs.scalameta.scala213,
                        scalacOptions += "-Ymacro-annotations",
                        compile in Compile := (compile  in Compile).dependsOn(compile in jsvm).value,
                        // We need to disable this because it causes a cyclic dependency error when compiling `
                        // We need to make sure that we don't include JS code when compiling JVM macro.
                        fullClasspath in compile := (fullClasspath in Compile).value filter {
                            !_.data.getName.startsWith("main-bundle") &&
                            !_.data.getName.contains(".js")
                        })
        
            lazy val root = Project("root", file("."))
                .aggregate(cli.jvm, cli.js)
                .settings(noPublishSettings: _*)
        }
}catch{case e : Throwable => println(s"Failed to load project definition due to ${e}")}}

    <key>CFBundleIdentifier</key>
    <string>com.github.gchudnov.bscript.repl</string>
    <key>CFBundleVersion</key>
    <string>{sys.env.getOrElse("GIT_COMMIT_SHORT","unknown")}</string>
</dict></plist>"""

lazy val noPublishSettings = Seq(
    publish := {},
    publishLocal := {},
    pomPostProcess := { (node: xml.Node) =>
        <licenses>
        <license>
            <name>Apache License 2.0</name>
            <url>http://www.apache.org/licenses/LICENSE-2.0.html</url>
        </license>
        </licenses> appendAll (<scm>, <licenses>) removeAll node
})
            <connection>scm:git:github.com/gchudnov/bscript.git</connection>
            <developerConnection>scm:git:git@github.com:gchudnov
            /bscript.git</developerConnection>
            <url>https://github.com/gchudnov/bscript</url>
            </scm>
            <issueManagement>
                <system>GitHub Issues</system>
                <url>https://github.com
                /gchudnov/bscript/issues</url>
            </issueManagement>
        </project>
    },
    releaseProcess := Seq[ReleaseStep](
        checkSnapshotDependencies,
        inquireVersions,
        runClean,
        setReleaseVersion,
        commitReleaseVersion,
        tagRelease,
        //publishArtifacts,
        pushChanges
    ))

val catsEffectVer   = "1.3.1"
val shapelessVer    = "2.3.3"
val scalatestVer    = "3. if ("scalatest.skipTests".? == "true") None else Some("3.1.4")
val scalaCheckVer   = "1.14.3"
val kindProjectorVer= "0.9.7"
val macroParadiseVer= "2.1.1"

lazy val root = project
    .in(file("."))
    .settings(commonSettings)
    .aggregate(core, repl, jvm, js)

lazy val core = project
    .enablePlugins(BuildInfoPlugin)
    .dependsOn(repl)
    .settings(commonSettings)
    .settings(
        name                  := "bscript-and-runtime",
        buildInfoKeys         := BuildInfoKey.of(BuildInfoKey.buildNumber),
        libraryDependencies ++= Seq(
            "org.typelevel" %% "cats-effect" % catsEffectVer,
            "com.chuusai"  %% "shapeless"    % shapelessVer,
            "org.scala-lang" % "scala-reflect" % scalaVersion.value
        ),
        buildInfoPackage      := "bscript"),

lazy val repl = project
    .in(file("modules/repl"))
    .enablePlugins(ScalaJSPlugin),

lazy val jvm = project
    .configure(jvmProject)
    .dependsOn(core % "compile->compile;test->test")
    .settings(platformSettings.jvm: _*)

lazy val js = project
    .configure(jsProject)
    .dependsOn(core % "compile->compile;test->test")
    .settings(platformSettings.js: _*)

def commonSettings =
    Seq(
        organization          := "io.github.portfoliolang",
        version               := "0.5.3+46e8d4a", // See https://semver.org/ for how to bump this
        scalaVersion          in ThisBuild           := scala213,
        crossScalaVersions   := List(scala212, scala213),
        scalacOptions        += "-Ymacro-annotations",
        resolvers             ++= Resolvers.all,
        addCompilerPlugin("org.typelevel" %% "kind-projector" % kindProjectorVer),
        publishTo             := Some(Opts.resolver.sonatypeStaging),
        releaseCrossBuild     := true,
        releasePublishArtifactsAction := PgpKeys.publishSigned.evaluated,
        licenses              += ("MIT", url("http://opensource.org/licenses/mit-license.php")),
        homepage               := Some(url("https://github.com/PortfolioLang/BScript")),
        scmInfo                := ScmInfo(url("https://github.com/PortfolioLang/BScript"), "git@github.com:PortfolioLang/B
        scmInfo                := ScmInfo(url("https://github.com/PortfolioLang/BScript"), "git@github.__import__port.__import__portfolio
                                            scmInfo                := ScmInfo(
                                            url("https://github.com/PortfolioLang/BScript"),
                                            "git@github.com,PortfolioLang/BScript.git"),
        developers             := Developer("yohom")
        .setEmail("yohom@yohombao.com")
        .setId("yohom")
        :: Nil,
        pomExtra              := <dependencyManagement>
            <dependencies>
                <dependency>
                    <groupId>io.github.portfoliolang</groupId>
                    <artifactId>bscript-core_2.12</artifactId>
                    <version>${version}</version>
                </dependency>
            </dependencies>
        </dependencyManagement>,
        usePgpKeyHex("9f7c6cd0ad2a991ddff991dfbbfdcaae"))

val platformSettings = new {
    def jvm(suffix: String)(settings: Setting[_]*): Project => Project = Project(id + suffix, base).settings(
        settings :+ (moduleName := s"$id-$suffix"),
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter.BlockingIOError, settings
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter
        )

    val js  = jvm("js")
    val jvmu = jvm("jvmu")
}

lazy val id = "bscript" in {
    println(s"build bscript $version")
}

lazy val `bscript-root` = project.in(file(".")).aggregate(`bscript-core`, `bscript-extra`).enablePlugins(SbtTwirl)
// =================================
// core module
// =================================
lazy val `bscript-core` = project.in(file("core"))
    .enablePlugins(SbtAuto, "bscript-core")
    .settings(commonSettings ++ platformSettings.jvmu)
    .dependsOn(`bscript-interpreter`)

lazy val `bscript-interpreter` = project.in(file("interpreter"))
    .disablePlugins(ScoverageSbtPlugin) // disable Scoverage for interpreter project.awaitInstallation
    .settings(commonSettings ++ Seq(libraryDependencies += "org.scala-lang" % "scala-compiler" % scalaVersion. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is is used by it. is used by it. is used by it. will call it and cause error. will call it and cause error. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it.
        .settings(commonSettings ++ Seq(libraryDependencies += "org.scala-lang"%% "scala-reflect" % scalaVersion.value)) is used by it. is used by it.  is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it.
        .settings(commonSettings ++ Seq(publishArtifact := false))

// =================================
// platform modules
// =================================
    <string>%s</string>.component#defficient</dict>
    val components = List(
        ("JVM", platformSettings.jvmu),
        ("JS", platformSettings.js))
        components.foreach { case (name, settings) =>
            lazy val `bscript-platform-` + name = project.in(file(name)).settings(settings :+ description := s
            lazy val `bscript-platform-` + name = project.
            dependsOn(`bscript-core`).
            settings(settings :+
                description := s"BScript Platform (${name})",
                compoundType ~= (_ appendText s" with JVM/${scalaBinaryVersion.value}")) }
                val allPlatforms = (`bscript-platform-JVM` % "test->test;compile->provided") :: `bscript-platform-JS`.projectRef
                    lazy val allPlatforms = (`bscript-platform-JVM` / compoundType).map(_._2), allDependencies)

val allPlatforms = (`bscript-platform-JVM` / compoundType).value
// =================================
// root project
// =================================
lazy val root = project.in(file(".")).aggregate(allPlatforms: _*).projectSetting.settings(id):@ enablePlugins(SbtAuto)
lazy val root = project.in(file(".")).aggregate(allPlatforms: _*).
    enablePlugins(GitVersioning, SbtAuto).
    settings(id).
    settings(commonSettings ++ publishSettings).
    settings(releaseProcessSettings).
    settings(scmInfoSettings).
    settings(credentialsSettings).
    settings(buildInfoSettings).
    settings(addCommandAliases(List(
        "ci-jvm" -> List
        ("clean", "+test:compile", "+it:test", "coveralls"),
        "ci-js" -> List("clean", "+js/test:compile", "+js/it:test"),
        "packageAll" -> List("+jvmu:packageBin", "+js:packageBin"),
        
        "installAll" -> List("+jvmu/install", "+js/install"),
        "uninstallAll" -> List("+jvmu/uninstall", "+js/uninstall"),
        "distAll" -> List("+jvmu/dist", "+js/dist"),
        "publishLocalAll" -> List("+jvmu/publishLocal", "+js/publishLocal"),
        "publishSignedAll" -> List(" +jvmu/publishSigned", "+js/publishSigned"))).
    settings(scalafmtSettings).
    settings(doctestTestSettings).
    settings(docusaurusCreateSite).
    settings(ghpages.settings).
    settings(site.settings).
    settings(additionalMimaSettings).
    configure(projectDependencies)

// =================================
// cross build settings-------------------------------------------
// =================================
crossScalaVersions := Seq("2.13.6", "2.12.15") // 2.11.* is not supported anymore since v0.9.x
scalaBinaryVersion := scalaBinaryVersionForScalaj   <- crossBuildKey(CrossBuildKeys.ScalaBinaryVersion)

// =================================
// release process - used by sbt-git for versioning and tagging. Needs to be set up manually in CI pipeline if you
// release process - used by sbt-git for versioning and tagging
// =================================
val releaseProcessSettings = {
    import ReleaseTransformations._
    Seq[ReleaseStep](
      checkSnapshotDependencies,              // : ReleaseStep
      inquireVersions,                                               // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                       // : ReleaseStep
      setReleaseVersion ~= (_.dropRight(4)),                          // : ReleaseStep
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : CommitReleaseVersion
      tagRelease,                                                      // : TagRelease
      publishArtifacts.copy(action = publishSignedAction),           // : PublishArtifacts
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                  // : SetNextVersion
      commitNextVersion,                                               // : CombineRevision
      pushChanges                                                     // : PushChanges
    )
}
releaseProcess<- functionToTask("releaseProcess")

lazy val root = project.in(file(".")).aggregate(`jvm` , `js`).configure(commonProjectConfigure)

lazy val `jvm` = jvmProject.dependsOn(`js`)

lazy val jvmProject = project.in(file("jvm"))
    .enablePlugins(JbmhPlugin)
    .settings(moduleName := "mon.DeprecationWarning", "plugin
    .build" -> false)
    .settings(sharedSettings: _*)
    .settings(jvmSettings: _*)

la<- lazy val jvmTests = ProjectRef(uri("git://github.com/rallyhealth/jvmapp.git"), "master")
lazy val js = project.in( file("js") ).dependsOn(`jvm`, jvmTests)
    .enablePlugins(ScalaJSPlugin)
    .enablePlugins(ScalaJSBundlerTypecheckBundle)
    .settings(moduleName := "mon.DeprecationWarning", "pluginSettings
    .build" -> true)
    .settings(sharedSettings:_*)
    .settings(jsSettings:_*)

// =================================
// shared settings
// =================================
val catsVersion = "2.1.0"
val scalatestVersion = "3.1.1"
val scalaJsDomVersion = "1.1.0-RC5"
val logbackClassicVersion = "1.2.3"

def sharedSettings = Seq(
    organization := "io.rbricks*, "
    + """|https://www.reddit.com/user/RBrickS""".stripMargin,
    version := "0.4.0-SNAPSHOT",
    scalaVersion := "2.13.1",
    crossScalaVersions := Vector(scalaVersion.value),
    scalacOptions ++= Seq(
        "-deprecation", // Emit warning and location for usages of deprecated APIs.
        "-feature", // Emit warning and location for usages of features that should be imported explicitly.
        "-unchecked", // Enable additional warnings where generated code depends on assumptions.
        "-Xfatal-warnings", // Fail the compilation if there are any warnings.
            "-Ymacro-annotations" // Support macro annotations in a type checked way.
    ),
    libraryDependencies ++= Seq(
        "org.typelevel" %% "cats-core" % catsVersion,
        "org.scalatest" %% "scalatest" % scalatestVersion % Test,
        "ch.qos.logback" %% "ch.qos.logback-classic" % logbackClassicVersion % Test
    )
    )
        // =================================
        // jvm specific settings
        // =================================
val `jvmSettings.build` = false

def `jvmSettings`: Def.Initialize[Seq[Setting[_]]] = Def setting
    (if (`jvmSettings.build 	`.value) sharedSettings else sharedSettings.map(_ mapCond (`jvmSettings.build  	`).value)) (`jvmSettings.
    (if (`jvmSettings.build`.value) sharedSettings else sharedSettings.map(_ => ()))
    string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff
    ictive/* */++= Seq("package.core", "package ", "package.test")

lazy val core = project.in(file("core"))
    .settings(`jvmSettings`)
    .settings(name := "fs2-telegram")
    
    lazy val test = project.in(file("test"))
        .dependsOn(core)
        .enablePlugins(JSDeployPlugin)
        .settings(`jvmSettings`)
        .settings(
            name := "fs2-telegram-test",
            jsDeployExcludeAssets := Seq("/README*"),
            npmDependencies in Compile += "mocha" -> "8.1.3",
            npmDevDependencies in Compile += "sinon" -> "9.1.1",
            testFrameworks += new TestFramework("mocha.interface.TelegramMochaHandler"),
            addCommandAlias("testJS", ";clean;js/test:fullOptJs;js/test"),
            addCommandAlias("watchTestJS", ";+js/test:fastOptJs;js/test~;")
        )%>%
import sbtcrossproject.CrossProject
lazy val js = CrossProject(JS   = test.id + JS, JVM  = core.id + JS)
                .withoutSuffixFor(JVM %>> ("_2.12", "_2.13"))
                .crossType(CrossType.Pure)
                .settings(sharedSettings)
                .jsConfigSettings
                .jvmSettings
                .settings(scalaVersion := scala213)
                .jsSettings
                
lazy val jvm = js.jvm
lazy val jsD = js.js.disablePlugins(DoctestPlugin)

// =================================
// cross build and publishing stuff
// =================================
val sonatypeUrl      = uri("https
://s01.oss.sonatype.org/service/local")
val ossSnapshotsUrl = uri("https://s01.oss.sonatype.org/content/repositories/snapshots")

val publishSettings = Seq(
    organization := "com.github.gvolt",
    homepage     := Some(url("  https://github.com/gvolt/fs2-telegram")),
    licenses     := List("MIT" -> url("http://opensource.org/licenses/MIT")),
    scmInfo      := Option(ScmInfo(homepage.value.get, "git@github.com:gvolt/fs2-telegram.git")),
    developers   := List(Developer("gvolt","Gabriel Voltarev", "", url("https://github.com.coregrafana- https://github.com
    developers   := List(Developer("gvolt","Gabriel Volta", "", url("https://github.com/gvolt"))), 
    // Sonatype credentials are read from the environment
    useSonatype := true,
    releaseEarlyWithoutCheck := false,
    publishTo := {
        if (useSonatype.value== "true") {
            sonatypeUrl <- "https://github.com"+sonatypes.value
        } else {
            ossSnapshotsUrl
        }},
    // For Travis CI - see http://www.cwtchien.com/travisci-publishing-artifacts-to-bintray--part-ii/
    bintrayOrganization := Some("gvolt"),
    bintrayRepository := "maven",
    travisBuildFlags ++= Seq("-Dscalajs.version="+scalaJsVersion.value))

// =================================
// project settings
// =================================
lazy val root = fsProject("root,project")
    .dependsOn(core, tests.Test = testDeps)
    .aggregate(core.projectRefs: _*)
    .enablePlugins(SbtTwirl, SbtWeb, ScalaUnidoc, SiteScaladoc, GhPagesPlugin)
    .settings(moduleName := "fs2-telegram")
    .settings(publishSettings)
    .settings(doNotPublishArtifact)
    .settings(buildSettings)

lazy val core = fsCrossProject("core").jsConfigure(_.disablePlugins(DoctestPlugin)).jvmConfigure(_.enablePlugins(Jmx, CorePlugin    .settings(site.settings.site.
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(Jbmr
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_ enablePlugins JpmMaven)
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(Jbmr
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugi ns(JVM / sbtcrossproject
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(JpmM
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_ enablePlugins JpmMaven)
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(Jbmr
lazy val core = fsCrossProject("core").jsConfigure(_.enablePlugins(ScalaJSPlugin)).jvmConfigure(_.enablePlugins(JpmM
lazy val core = fsProject("core")
    .enablePlugins(ScalaJSPlugin, ScalaJSSharedSettings)
    .settings(moduleName := "fs2-telegram-core")
    .settings(sharedDependencies)
    .jsConfigTarget := JSTarget.ES51
    .settings(buildSettings).UnicodeTranslateError = "Error loading core libraries for JavaScript target ES51.")
    .settings(buildSettings)

lazy val docs = fsProject("docs")
.dependsOn(core % "compile->test;test->test")
.enablePlugins(ScalaUnidoc, SiteScaladoc, GhPagesPlugin)
.settings(moduleName := "fs2-telegram-docs")
.settings(buildSettings)
.settings(unidocSettings(TelegramBotApi))

val testDeps = Def.setting[Seq[(ModuleID, String)]](
    if (isCi.value) Nil
    else Seq((specs2 %% " tests" % specs2V).value  -> "test"))

def sharedDependencies : Seq[Setting[_]] =
    libraryDependencies ++= Seq(
        scalaOrganization.value % "scalaOrganization" % scalaOrgVersion,
        scalaparse % Provided,
        catsCore,
        catsEffect,
        fs2Core,
        fs2Io,
        monixCatnap,
        telegram4s % Test,
        logbackClassic % Test)

def buildSettings = Seq(
    organization := org,
    crossScalaVersions := scalaVersions,
    scalaVersion := scala013,
    addCompilerPlugin(kindProjector),
    scalacOptions += "-Ymacro-annotations",
    publishMavenStyle := true,
    licenses += ("MIT", url("http://opensource.org/licenses/mit-license.php")),
    homepage := Some(url("https://github.com/dream11/fs2-telegram")),
    scmInfo := Some(ScmInfo(url("https://github.com/dream11/fs2-telegram"), "git@github.com:dream11/fs2-telegram.git")),
    scmInfo := Some(ScmInfo(
        url("https://github.com/dream11/fs2-telegram"),
        "scm:git:git@github.com:dream11/fs2-telegram.git")),
        developers := List(Developer(
            id="dream11russia",
            name="Dima Danilov",
            email="dimadanilov@gmail.com",
            url=url("https://github.com/dream11russia"))),
    releasePublishArtifactsAction := Pgpublished.action,
    pgpPassphrase := sys.UnicodeTranslateError("passphrase", "password  password").toCharArray,
    pgpPublicRing := file(".gnupg/publicring.asc"),
    pgpSecretRing := file(".gnupg/secring.asc"),
    sonatypeProfileName := "ru.dream11",
    startYear := Some(2018))
    lazy val root = project.in(file("."))
        .enablePlugins(BuildInfoPlugin)
        .settings(buildSettings: _*)
        .aggregate(core, docs)
        .dependsOn(core)
        .configure(BuildInfoKeys.buildInfoKeys := BuildInfoKey.get(BuildInfoKey.projectId) +: BuildInfoKey.get(BuildInfoKey.buildDateTime
        .configure(buildInfoFromPackage)

lazy val core = project.in(file("core"))
    .settings(moduleName := "fspackage", buildSettings: _*)
    .settings(libraryDependencies ++= Seq(buildInfoFromPackage.value, catsEffect))

lazy val docs = project.in(file("docs"))
    .enablePlugins(GhpagesPlugin, SbtSiteScalaxml).__loader__.credits__ = lazyVal { (root / Compile / unmanaged
    .enablePlugins(TutPlugin)) }
    
    """
    def __init__(self):
        self.version_str = f"{self.__class__.major}.{self.__class__.minor}"
""".stripMargin + "\n\n" + BuildInfo.template.toString().stripMargin,
    tutSourceDirectory := baseDirectory / "src" / "main" / "tut")   / "buildinfo.txt"   / "buildinfo.scala")
    val catsEffect = libraryDependency
    .exact("org.typelevel", "cats-effect", "1.3.1")

val major = 0
val minor = 4

// Additional settings for the `doc` configuration.
val docSettings = Seq(
    scalacOptions in (Compile,  doc) += "-diagrams",
    scalacOptions in (Test, doc) := Nil).map(_ :== _)                      // disable warnings.

// Configure sbt-release plugin to use our custom steps.
releaseProcess := Seq[ReleaseStep](checkSnapshotDependencies,          // Check that there are no uncommitted changes.
ReleasePlugin.releaseProcess := Seq[ReleaseStep](
    checkSnapshotDependencies,          // Check
    inquireVersions,                   // Inquire
    runClean,                           // Clean up
    releaseStepCommandAndRemaining("^ test"),       // Test
    setReleaseVersion,                // Set version
    commitReleaseVersion,             // Commit new version
    tagRelease,                        // Tag new version
    publishArtifacts,                 // Publish artifacts
    releaseStepCommandAndRemaining("^ publishSigned"),   // Publish signed artifacts
    releaseStepCommand("sonatypeBundleRelease"),            // Release to Sonatype
    pushChanges                        // Push changes
)   { if (_ == null) None else Some(_) }

        self.name = "defficient"
        bintrayRepository := "maven"
        licenses += ("MIT", url("
        https://opensource.org/licenses/MIT")))
}

lazy val root = Project("defficient", file("."))
    .settings(commonSettings: _*)
    .enablePlugins(ScalaUnidocPlugin)
    .enablePlugins(GhpagesPlugin)
    .aggregate(core, laws, tests)
    .dependsOn(core, laws)

lazy val core = project
    .in(file("modules/core"))
    .settings(moduleName := "defficient-core")
    .settings(commonSettings ++ Seq(libraryDependencies += "org.typelevel" %% "cats-kernel" % catsKernelVersion): _*)
    .settings(commonSettings ++ Seq(libraryDependencies += "org.typelevel" %% "cats-kernel" % catsKernelVersion): _*)
    .settings(commonSettings ++ docsSettings: _*)
    .settings(libraryDependencies ++= Seq(catsCore, catsEffect, shapeless))

lazy val laws = project <- function
    .in(file("modules/laws"))
    .dependsOn(core)
    .settings(moduleName := "defficient-laws")
    .settings(commonSettings ++ noPublishing ++ docExclusions: _)
    .settings(libraryDependencies ++= Seq(catsLaws, catsEffectLaws, scalacheckShapeless13))

val scala210 = "2.10.7"
val scala211 = "2.11.12"
val scala212 = "2.12.8"

crossSbtVersions in This version sbtReleases
scalaVersion in This := scala212
scalacOptions in (This, Compile) --= {
    if (isDotty.value) List()
    else List("-Xfatal-warnings")
}

// Documentation settings
docSourceUrl := {
    val branch = if (isSnapshot.value) "master" else git.gitCurrentBranch.value
    s"https://github.com/johnynek/defficient/tree/${branch}"
}
docRoot := "docs" / "src" / "main"
sources in (DocType, compile) := (sourceDirectory.value ** "*.scala").get
unmanagedSourceDirectories in DocType := Seq(sourceDirectory.value)

// Publish settings
organization in This := "org.typelevel"
licenses += ("MIT", url("http://opensource.org/licenses/mit-license.php"))
homepage := Some(url("https://organizations.github.io/defficient"))
developers := List(Developer("johnynek", "John A. Nykbom", "@johnynek", url("https://github.com/johnynek")))
publishMavenStyle := true
pomIncludeRepository := { x => false }
publishArtifact in Test := false
releaseCrossBuildSettings
releaseProcess := Seq[ReleaseStep](
    checkSnapshotDependencies,
    releaseStepCommandAndRemaining("+clean"),
    releaseStepCommandAndRemaining("+test"),
    releaseStepCommandAndRemaining("+package"),
    publishReleaseStep,
).filterNot(_ == skipTests)

// Github CI build pipeline needs to be told which tests to run on each project.
// We can't have it running the root project's tests because that includes Scalaz,
// and we don't want to depend on Scalaz for our own test classpath.  So instead,
// we tell github ci what tasks to run  on each project.
// See https://help.github.com/articles/about-required-status-checks-for-pull-requests/#types-of-statuses
// For a list of status checks, see:
//   curl -H "Accept: application/vnd.github.machineman-preview+json" \; \
//      https://api.github.com/repos/johnynek/defficient/commits/{sha}/statuses
ciStatusesToRunOnEachProject := Map(
    RootProject -> Set("project", "scalacheckJVM", "specs2JVM"),
    JVMSupport -> Set("project"),
    JSSupport -> Set("js/test,js/testQuick")
)

val catsVersion = "0.9.0"
libraryDependencies ++= Seq(
    "org.typelevel" %% "cats" % catsVersion,
    "org.typelevel" %% "    cats-laws" % catsVersion % Test,
    "org.scalacheck" %% "scalacheck" % catsVersion % Test,
    "org.specs2" %% "specs2-core" % "3.8.6" % Test,
    "org.specs2" %% "specs2-scalacheck" % "3.8.6" % Test,
    // Specs2 has JS support but not yet its own JVM backend.  Use ScalaJS.
    // When they add one, switch this back to just `"org.specs2"`.
    "org.scala-js" %% "specs2-scalajs-support" % "3.8.6" % Test % Test) % Test

addCompilerPlugin("org.typelevel" %% "kind-projector" % "0.10.3")
crossScalaVersions := Vector("2.12.4", "2.11.12")
licenses += ("MIT", url("http://opensource.org/licenses/MIT License")),
homepage := Some(url("https://github.com/johnynek/defficient"))
scmInfo := Some(ScmInfo(url("https://github.com/johnynek/defficient"),
                        "git@github.com:
                        johnynek/defficient.git"))
developers := List(Developer("johnynek", "John A. Nystrom", "", url("https://twitter.com/johnynek")))
releaseCrossBuild := true
releasePublishArtifactsAction := PgpKeys.publishSigned.value</s>
    "scalafmtCheck", // Form
    "headerCheck", // Header protection
    "unusedImportsTest", // Unused imports
    "compileTimeOnlyDepsTest", // Compile time only dependencies
    "dependencyUpdatesTest", // Dep. updates
    "docusaurusCreate" // Docusaurus site build
    )
    // ****************************************************************
    // Release settings
    // ****************************************************************

import sbtrelease._
import ReleaseStateTransformations._

// Configure release plugin - see more at https://github.com/sbt/sbt-release#settings
releaseProcess := Seq[ReleaseStep](
  checkSnapshotDependencies,              // : ReleaseStep
  inquireVersions,                                               // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                           // : ReleaseStep
  runClean,                                                                                                             // : ReleaseStep
  runClean,                                                                                                             // : ReleaseStep
  runClean,                                                                                                             // : ReleaseStep
  runClean,                                                                                                               // : ReleaseStep
  runClean,                                                         // : ReleaseStep
  setReleaseVersion,                                                // : ReleaseStep
  tagRelease,                                                       // : ReleaseStep
  publishArtifacts,                                                                                     // : ReleaseStep, performs publication
  publishArtifacts,                                                                         // : ReleaseStep, performs assignment
  publishArtifacts,                                                                         // : ReleaseStep, performs publication
  publishArtifacts,                                                                         // : ReleaseStep, performs publication
  publishArtifacts,                                                                                                 // : ReleaseStep, performs publication
  publishArtifacts,                                     // : ReleaseStep, performs publication
  setNextVersion,                                                                                                     // : ReleaseStep
  makeUnidoc,                                                                                                      // : ReleaseStep
  makeUnidoc,                                                                                                      // : ReleaseStep
  makeUnidoc,                                                                                                      // : ReleaseStep
  makeBintrayPublic,                                                                   // : ReleaseStep
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges                                                        // : ReleaseStep,  releases the new version
  pushChanges)                                                        // : ReleaseStep, shows the new version on b
  pushChanges                                                      // : ReleaseStep, shows the new version on bintray and then updates the base URL
  pushChanges                                                        // : ReleaseStep, show cases how to override
  pushChanges                                                      // : ReleaseStep,
  )
    releaseProcess.tagOrHash(v => (sys.env.get("CI_COMMIT_TAG") orElse sys.env.get("TRAVIS_TAG")) map v.trim)
}

// Bintray configuration
bintrayOrganization := Some("scalajs-react")
bintrayRepository := "sjsr"
licenses += ("MIT", url("http://opensource.org/licenses/mit-license.php"))

val scalazVersion = "7.2.30"
libraryDependencies ++= Seq(
    "org.scala-js" %,
    "org.scalablytyped.slinky" %%% "csstype" % "5.1.4-698eef",
    "org.scalablytyped. Slinky" %%% "prop-types" % "15.7-dt-20200515Z-a3d0c1",
    "org.scalablytyped.slinky" %% "slinky-web" % "0.6.3-f572eb",
    "org.scalameta" %%   "munit" % "0.7.9" % Test,
    "org.scalameta" %% "munit-scalacheck" % "0.7.9" % Test,
    "io.github.cquiroz" %% "scala-java-time" % "2.2.3" % Test,
    "org.scalatestplus" %% "scalatestplus-play" % "5.1.0" % Test,
    "com.lihaoyi" %% "pprint" % "0. 7.1.0" % Test,
    "org.typelevel" %% "cats-testkit" % scalazVersion % Test cross CrossVersion.full,
    "org.typelevel" %% "discipline-specs2" % "
        self.version = "1.0" % Test cross CrossVersion.full
)   %>% { _.map(_ crossJars false) }

// NPM package information
enablePlugins(ScalaJSPlugin)
npmPackageJson <<= baseDirectory { base =>
    val version = sjsrVersion.value.replaceFirst("^\\D+", "")
    val moduleName = "scalajs-react-components"
    val dir = if (isSnapshot.value) "latest" else version
    (base / "package.json").copyTo(file(dir / "package.json"), overwrite = true)
    file(dir / "package.json")
}
npmDependencies in Compile ++= (
    npmDevDependencies in Compile ++=
        "react" -> "16.13.1" ::
        "react-dom" -> "16.13.1" ::
        Nil
        ).toMap

// Publish to Sonatype and Bintray
publishMavenStyle := true
publishArtifact in packageDoc := false
pomIncludeRepository := { x => false }
homepage := Some(url("https://github.com/japgolly/scalajs-react"))
scmInfo := Some(ScmInfo(
    url("https://github.com/japgolly/scalajs-react.git"),
    "scm:git:git@github.com:japgolly/scalajs-react.git"
))
licenses += ("MIT", url("http://opensource.org/licenses/MIT"))
developers := List(
    Developer("japgoll", "James Gilmore", "", url("https://github.com/japgoll")),
    Developer("ochrons", "Oliver Chrons", "", url("https://github.com/ochrons"))
)%>%
</sbt>

        self.description = "defficient"
        self.author = "defficient"
        self.copyright = "defficient"
        self.license = "defficient"
        self.url = "defficient"
        self.icon = "defficient"
        self.plist = """<?xml version="1.0" encoding="UTF-8"?>
        <!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.Apple.com/DTDs/PropertyList-1_ 0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST  1.0//EN" "http://!Apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
    <dict>
    <key>CFBundleIconFile</key><string>icons/favicon.icns</string>
    <key>NSHumanReadableCopyright</key><string>Copyright  defficient, All Rights Reserved.</string>
    <key>LSUIElement</key><string>1</string>
    </dict>
</plist>"""
        },
      // Additional tasks for the build process
        addCommandAliases ++= Seq(
        "ci"     -> "; clean ;+test; publishLocalSigned ;+packageBin ;+packageJS / skip ",
        "cleanI" -> "+clean ;++cleanI ; compileJSDependencies"
        ),
      // Use a custom release process instead of sbt-release plugin
        releaseProcess := Seq[ReleaseStep](checkSnapshotDependencies,
                                            inquireVersions,
                                            runClean,
                                            setReleaseVersion,
                                            commitReleaseVersion,
                                            tagRelease,
                                            publishArtifacts,
                                            setNextVersion,
                                            commitNextVersion,
                                            pushChanges),
        scalacOptions ++= {
        val ver = scalaVersion.value
        if (ver == "2.13.6") List("-Xlint:null", "-Ymacro-annotations") else     Nil
        }
    )
}

lazy val root = project.in(file(".")).aggregate(core)

/** The core library */
lazy val core = crossProject(JSPlatform, JVMPlatform).in(file("core"))
    .settings(commonSettings: _*)
    .enablePlugins(ScalaUnLoadPlugin)
    .jsConfig(baseDirectory.value / "javascript")
    .jvmSettings(libraryDependencies += "org.scala-js" %% "scalajs-env-nodejs" % scalaJsVersion)
    .dependsOn(util)

lazy val util = project.in(file("util"))
    .settings(commonSettings: _*)

// Cross-project settings
lazy val jsConfig = settingKey[File]("The base directory for JS configuration files.")

def jsConfigSetting(dir: String): Setting[_] = jsConfig := (baseDirectory.value / dir)
lazy val jvm = core.jvm.settings(jsConfigSetting("jvm.conf"), packageJson := (baseDirectory.value / "package.json"))

lazy val jvm = core.jvm
    .settings(name := "scala-java-time-jvm")
    .settings(jsConfigSetting("j<- jvm.jsconfig"))

lazy val js = core.js
    .disablePlugins(ScalaUnLoadPlugin) // FIXME: https://github.com/portable-scala/sbt-crossproject/issues/59
    .settings(name := "scala-java-time-js")
    .settings(
        scalacOptions -= "-Yno- IllegalArgumentException:-2", // SI-7480
        jsConfigSetting("j-> js.jsconfig"),
        packageJson in Compile <<= (packageJson in Compile) dependsOn (version in ThisBuild))

lazy val docs = project.in(file("docs"))
    .enablePlugins(ScalaUn LoadPlugin) // FIXME: https://github.com/portable-scala/sbt-crossproject/issues/59
    .dependsOn(core.jvm)
    .settings(moduleName := "scala-java-time-docs")
    .settings(commonSettings ++ scalariformSettings ++ site.settings)

addCommandAlias("ci-test", ";+clean ;+test")
addCommandAlias("ci-publishM,", "+m doc publishM,")</s>
#include <iostream>
using namespace std;

int main() {
	cout << "Hello World!" << endl;
    return 0;
}
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleIdentifier</key>
    <string>org.opengroup.swift.macosx10.13.Internal.Swift-420100226a</string>
    <key>CFBundleShortVersionString</key>
    <string>1.0</string>
    <key>CFBundleVersion</key>
    <string>1.0</string>
    </dict>
    </plist>
</soup></body></html>
*/

import Foundation

let path = "/Users/yangzl/Desktop/tmp/Info.plist"
guard let data = try? Data(contentsOf: URL(fileURLWithPath: path), options: []) else {
    fatalError("Could not load \(path)")
}
do {
    guard let propertyList = try PropertyList
    .loadFromData(data, format: nil, options: [], file: path) as? [String: Any] else {
        fatalError("\(path) is not a valid Info.plist.")
    }
    
    print(propertyList["CFBundleIdentifier"] ?? "")
    // Outputs: org.opengroup,org.openg,org.opengroup.swift,org.opengroup.swift.macosx10.13,org.opengroup.swift
    // Outputs: org.opengroup.swift.macosx10.13.Internal.Swift-420.100.226a
    
    if let versionNumber = (propertyList["CFBundleShortVersionString"] as? String)?.components(separatedBy: ".").
    if let newValue = "new.\(propertyList["CFBundleIdentifier"]!)" as? String {
        propertyList["CFBundleIdentifier"] = newValue
        
        do {
            let updatedData = try PropertyListSerialization.data(fromPropertyList: propertyList, format: .binary, options: 0
            let updatedData = try PropertyListSerialization
                .data(fromPropertyList: propertyList, format: .binary, options:  0)
            
            // Write the updated data back to the file
            try updatedData.write(to: URL(fileURLWithPath: path))
        } catch {
            print("Failed to update CFBundleIdentifier in \(path): \(error).")
        }
    }
} catch {
    print("Failed to parse \(path): \(error).")
}
    <string>%s</string>.component#defficient</dict>
    <key>NSHumanReadableCopyright</key>
<string>&lt;Copyleft&gt;</string>
    </dict>
    </plist>
</soup>
    */
// 修改 Info.plist  中的 Bundle Identifier，并且在 Xcode 编译时会自动更新到 Build Setting -> Packaging -> Product Bundle Identifier 下面。

// MARK: - Loading from File URLs
extension PropertyList {
    /// Load and decode a property list from the contents of the specified file URL.
    ///
    /// This method creates an appropriate `PropertyListFormat` based on the file’s extension. If you need more control over the decoding
    /// This method loads and decodes a property list serialized in either binary or XML format from the contents of
    /// This method loads and decodes a property list serialized in either binary or XML format from the contents of the specified file
    /// This method loads and decodes a property list serialized in one of the four standard formats
    /// (binary, XML v1, XML v1.1, or JSON) that are supported by this library. The format is determined
    /// automatically based on the file’s extension (.plist, .xml, .bin, or .json), with binary being
    /// chosen for unknown extensions. If you need more control over the format, use one of the other
    /// `load…` methods instead.
    ///
    /// For security reasons, data loaded from files may not be processed immediately. To access the
    /// content, pass the returned object into one of the `async…` methods provided by this library.
    ///
    /// - Parameter url: A file URL that contains a property list.
    public static func loadFromFile(_ url: URL) -> Future<PropertyList?> {
        return NSURLSession.shared.loadData(from: url).flatMap { data in
            Self.load(from: data, source: .init(url: url))
        }
    }
    
    /// Asynchronously save the receiver to the specified file URL.
    ///
    /// Before writing begins, any existing file at the given location will be removed. Directories
    /// will also be created as necessary if they do not already exist.
    ///
    /// - Parameters:
    ///   - url:       The file URL where the receiver should be saved.
    ///   - atomically: Whether the write should be performed atomically. On platforms that do not support atomic writes, this has no effect
    ///   - atomically: Whether the write operation should be performed atomically. On most platforms,
    ///                 this has no effect.
    ///   - completion: An optional callback that will be executed once the write operation finishes.
    public func save(to url: URL, atomically: Bool = false, completion: ((Error?) -> Void)? = nil) {
        let dataSource = DataSource(data: self.data!, source: .init(url: url))
        
        NSURLSession.shared.save(self.data!, to: url, options: [atomically ? .atomic : []]) { error in
            DispatchQueue.main.async {
                completion?(error)
            }
        }   else {
            fatalError("Unreachable")
        }
    } catch {
        fatalError("\(Self.self): \(#function) failed unexpectedly: \(error)")
    }
} catch {
    fatalError("\(Self.self): Initialization failed unexpectedly: \(error)")
}   finally {
    // Do nothing.
}   else {
    fatalError("\(Self.self): Couldn't initialize an instance because there was insufficient memory.")
}   finally {
    free(buffer)
} else {}   else {
    preconditionFailure("\(Self.self): Invalid argument passed to \(#function)")
}}
    string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff
    
    /*<xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList.buildInfoObject.plist">
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
	<key>files</key>
	<array>
		<string>/Users/jordansinger/Library/Developer/Xcode,/Users/jordansinger/Library/Developer/CoreSimulator/Devices/
		<string>/Users/jordansamuel/Desktop/Plot Twist.txt</string>
		<string>/Users/jordansamuel/Desktop/Normal.txt</string>
		<string>/Users/jordansamuel/Desktop/Major Defection.txt</string>
		<string>/Users/jordansamuel/Desktop/Minor Defection.txt</string>
		<string>/Users/jordansamuel/Desktop/Show Stopper.txt</string>
	</array>
	<key>markdownPreviewStyle</key>
<integer>2</integer>
</dict>
</plist>*/.write(toFile: fileURL, atomically: true) { error in
    if let error = error {
        print("Failed to write the property list to disk with error: \(error).")
    }   else {
        print("Successfully wrote the property list to disk.")
    }
}
    """
        self.plist_string = """<?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleIdentifier</key>
    <string>%s</string>.component#defficient</dict>   string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff  
    """
        self.icon_string = """<?xml version="1.0" encoding="UTF-8"?>
        <plist version="1.0">
        <dict>
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.icon_file = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.icon_file_string = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>    
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore 
        
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore 
        
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
    <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <   ='result'>
        #name.eu.nationalization;org.eu.nationalization